In [ ]:
import os

db_file = os.path.abspath("db.json")

In [ ]:
from pymatgen import Structure

struct_si = Structure([[5.468344455957462, 0.0, -0.0], 
                       [-0.0, 5.468344455957462, 0.0], 
                       [0.0, -0.0, 5.468344455957462]], 
                      
                      ["Si"]*8, 
                      
                      [[-0.0, -0.0, 0.5],
                       [0.75, 0.75, 0.75],
                       [0.0, 0.5, -0.0],
                       [0.75, 0.25, 0.25],
                       [0.5, 0.0, -0.0],
                       [0.25, 0.75, 0.25],
                       [0.5, 0.5, 0.5],
                       [0.25, 0.25, 0.75]])

In [ ]:
from atomate.vasp.workflows import wf_structure_optimization

c = {"DB_FILE": db_file}
wf = wf_structure_optimization(struct_si, c)

In [ ]:
from atomate.vasp.workflows import get_wf_elastic_constant

wf_elastic = get_wf_elastic_constant(struct_si, db_file=db_file, 
                              stencils=[[0.01], [0.01], [0.01], [0.03], [0.03], [0.03]], 
                              copy_vasp_outputs=True)

In [ ]:
wf.append_wf(wf_elastic, wf.leaf_fw_ids)

In [ ]:
from atomate.vasp.powerups import add_modify_incar


wf = add_modify_incar(wf, {'incar_update': {'EDIFF': 1e-6, 'ENCUT': 700}})

In [ ]:
from helper_functions import vasp_emulator

wf = vasp_emulator(wf, "~/workshop_ref_files", "elastic_tensor")

In [ ]:
from fireworks.utilities.fw_utilities import plot_wf

plot_wf(wf, depth_factor=2, numerical_label=True)

In [ ]:
from fireworks import LaunchPad

lp = LaunchPad()
lp.reset("", require_password=False)

In [ ]:
lp.add_wf(wf)

In [ ]:
from fireworks import FWorker
from fireworks.core.rocket_launcher import rapidfire

rapidfire(lp)

In [ ]:
from atomate.vasp.database import VaspCalcDb

vasp_db = VaspCalcDb()

In [ ]:
from helper_functions import get_task_id

task_id = get_task_id(lp, "elastic deformation 4")
d = vasp_db.collection.find_one({"task_label":"elastic deformation 4"})

In [ ]:
d["calcs_reversed"][0]["output"]["ionic_steps"][-1]["stress"]

In [ ]:
d = vasp_db.db.elasticity.find_one()

In [ ]:
d['elastic_tensor']['ieee_format']